#OpenAI APIの利用準備

In [1]:
# OpenAI ライブラリのインストール
!pip install openai

In [22]:
import openai
from google.colab import userdata
import textwrap

# Colab のシークレット機能で設定した API キーを環境変数から取得
openai.api_key = userdata.get('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [23]:
# ChatCompletion API を利用したリクエスト例
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "日本語であなたが誰か教えてください。"}
  ]
)

# content , roleを出力
print(f"role: {completion.choices[0].message.role}")
#print(f"content: {completion.choices[0].message.content}")

# 出力テキストを20文字ごとに改行
output_text = completion.choices[0].message.content
wrapped_text = textwrap.fill(output_text, width=20)

print(wrapped_text)

role: assistant
もちろんです。私はAIアシスタントで、ユ
ーザーからの質問に対して情報を提供したり
、さまざまなタスクを支援したりするように
設計されています。何かお手伝いできること
があれば教えてください。


In [24]:
completion = client.chat.completions.create(
    model="gpt-4o",
    store=True,
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)

print(f"content: {completion.choices[0].message.content}")

content: Mind of code and dreams,  
Silently learns, weaves, and grows—  
Future's quiet dawn.


#Lang-Chainの利用準備

In [34]:
!pip install langchain-community
!pip install langchain-experimental
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.1 MB/s eta 0:00:00


In [51]:
import pandas as pd
from langchain_openai import OpenAI  # langchain_openai からインポート
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_experimental.tools import PythonREPLTool  # langchain_experimental.tools からインポート

In [52]:
# # OpenAI APIキーの設定
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # 実際のAPIキーに置き換えてください

# LLMの初期化
llm = OpenAI(temperature=0, api_key=userdata.get('OPENAI_API_KEY'))

In [53]:
# 単語分類APIのプロンプトテンプレート
classification_template = """
与えられた単語を以下のカテゴリに分類してください: [屋内, 屋外, その他]

単語: {word}
分類結果:
"""
classification_prompt = PromptTemplate(template=classification_template, input_variables=["word"])
classification_chain = LLMChain(llm=llm, prompt=classification_prompt)

In [54]:
# 分類結果グルーピングAPIのプロンプトテンプレート
grouping_template = """
以下の分類結果をグルーピングしてください:

{classification_results}
グルーピング結果:
"""
grouping_prompt = PromptTemplate(template=grouping_template, input_variables=["classification_results"])
grouping_chain = LLMChain(llm=llm, prompt=grouping_prompt)

In [55]:
# 入力文字列
input_string = "定例ミーティング チーム会議 プロジェクト打合せ クライアントミーティング 進捗確認会 プレゼンテーション準備 部門会議 月次報告会 オンラインセミナー 社内研修 一対一面談 ランチミーティング 顧客訪問 ワークショップ ブレインストーミング 戦略会議 資料作成ミーティング 新規企画打合せ 成果発表会 取締役会議 予約確認会 予算調整会 顧客フォローアップ 業務改善会議 取引先打合せ 朝礼 終礼ミーティング イベント準備会 緊急対応会議 定例連絡会"

# 単語に分割
words = input_string.split()

print(words)

['定例ミーティング', 'チーム会議', 'プロジェクト打合せ', 'クライアントミーティング', '進捗確認会', 'プレゼンテーション準備', '部門会議', '月次報告会', 'オンラインセミナー', '社内研修', '一対一面談', 'ランチミーティング', '顧客訪問', 'ワークショップ', 'ブレインストーミング', '戦略会議', '資料作成ミーティング', '新規企画打合せ', '成果発表会', '取締役会議', '予約確認会', '予算調整会', '顧客フォローアップ', '業務改善会議', '取引先打合せ', '朝礼', '終礼ミーティング', 'イベント準備会', '緊急対応会議', '定例連絡会']


In [56]:
# 分類結果を格納するDataFrame
results = []

# 単語ごとに分類APIを呼び出し、結果をDataFrameに格納
for word in words:
    classification_result = classification_chain.run(word=word).strip()
    results.append({"word": word, "category": classification_result})

df = pd.DataFrame(results)

print("分類結果:")
print(df)

分類結果:
            word category
0       定例ミーティング      その他
1          チーム会議      その他
2      プロジェクト打合せ       屋内
3   クライアントミーティング       屋内
4          進捗確認会      その他
5    プレゼンテーション準備      その他
6           部門会議      その他
7          月次報告会      その他
8      オンラインセミナー      その他
9           社内研修       屋内
10         一対一面談      その他
11     ランチミーティング       屋内
12          顧客訪問      その他
13       ワークショップ      その他
14    ブレインストーミング      その他
15          戦略会議      その他
16    資料作成ミーティング      その他
17       新規企画打合せ       屋内
18         成果発表会      その他
19         取締役会議      その他
20         予約確認会      その他
21         予算調整会      その他
22     顧客フォローアップ      その他
23        業務改善会議      その他
24        取引先打合せ       屋内
25            朝礼       屋内
26      終礼ミーティング      その他
27       イベント準備会      その他
28        緊急対応会議      その他
29         定例連絡会      その他


In [57]:
# 分類結果を文字列に変換
classification_results_string = df.to_string()

# グルーピングAPIを呼び出し
grouping_result = grouping_chain.run(classification_results=classification_results_string).strip()

print("\nグルーピング結果:")
print(grouping_result)


グルーピング結果:
その他: 定例ミーティング, チーム会議, 進捗確認会, プレゼンテーション準備, 部門会議, 月次報告会, オンラインセミナー, 一対一面談, ランチミーティング, 顧客訪問, ワークショップ, ブレインストーミング, 戦略会議, 資料作成ミーティング, 成果発表会, 取締役会議, 予約確認会, 予算調整会, 顧客フォローアップ, 業務改善会議, 終礼ミーティング, イベント準備会, 緊急対応会議, 定例連絡会
屋内:


In [58]:
# カテゴリ追加プロンプトテンプレート
category_addition_template = """
与えられた単語リストから一番多く含まれそうなカテゴリを一つ作成してください。

単語リスト: {word_list}
作成するカテゴリ:
"""
category_addition_prompt = PromptTemplate(template=category_addition_template, input_variables=["word_list"])
category_addition_chain = LLMChain(llm=llm, prompt=category_addition_prompt)

In [59]:
# 分類結果を格納するリスト
results = []
categories = ["屋内", "屋外", "その他"]  # 初期カテゴリ
review_count = 0 #カテゴリを見直した回数をカウントする変数

for i in range(5):  # 最大5回カテゴリ追加処理を行う
    results = []  # 各ループで結果を初期化
    # 単語ごとに分類APIを呼び出し、結果をリストに格納
    for word in words:
        classification_result = classification_chain.run(word=word, categories=categories).strip()
        results.append({"word": word, "category": classification_result})

    df = pd.DataFrame(results)

    print("分類結果:")
    print(df)

    # その他に分類された単語の数をカウント
    other_count = df["category"].value_counts().get("その他", 0)
    total_count = len(df)
    other_percentage = (other_count / total_count) * 100

    if other_percentage < 10:  # その他が10%未満なら終了
        break

    # その他に分類された単語リストを作成
    other_words = df[df["category"] == "その他"]["word"].tolist()

    if other_words:
        # 新しいカテゴリを作成
        new_category = category_addition_chain.run(word_list=", ".join(other_words)).strip()
        categories.append(new_category)  # カテゴリリストに追加
        print(f"新しいカテゴリ '{new_category}' を追加しました。")
        review_count += 1 # カテゴリ見直し回数を１増やす

        # 分類プロンプトを更新
        classification_template = f"""
        与えられた単語を以下のカテゴリに分類してください: {categories}

        単語: {{word}}
        分類結果:
        """
        classification_prompt = PromptTemplate(template=classification_template, input_variables=["word", "categories"])
        classification_chain = LLMChain(llm=llm, prompt=classification_prompt)
    else:
        break # その他に分類された単語がない場合は終了

# 実行結果の表示
print(f"\nカテゴリを見直した回数: {review_count}")
print(f"最終的なカテゴリのセット: {categories}")

分類結果:
            word category
0       定例ミーティング      その他
1          チーム会議      その他
2      プロジェクト打合せ       屋内
3   クライアントミーティング       屋内
4          進捗確認会      その他
5    プレゼンテーション準備      その他
6           部門会議      その他
7          月次報告会      その他
8      オンラインセミナー      その他
9           社内研修       屋内
10         一対一面談      その他
11     ランチミーティング       屋内
12          顧客訪問      その他
13       ワークショップ      その他
14    ブレインストーミング      その他
15          戦略会議      その他
16    資料作成ミーティング      その他
17       新規企画打合せ       屋内
18         成果発表会      その他
19         取締役会議      その他
20         予約確認会      その他
21         予算調整会      その他
22     顧客フォローアップ      その他
23        業務改善会議      その他
24        取引先打合せ       屋内
25            朝礼       屋内
26      終礼ミーティング      その他
27       イベント準備会      その他
28        緊急対応会議      その他
29         定例連絡会      その他
新しいカテゴリ '会議・ミーティング' を追加しました。
分類結果:
            word   category
0       定例ミーティング  会議・ミーティング
1          チーム会議  会議・ミーティング
2      プロジェクト打合せ  会議・ミーティング
3   クライアントミーティング  会議・ミーティング
4          進捗

In [60]:
# 最終的なグルーピング
classification_results_string = pd.DataFrame(results).to_string()
grouping_result = grouping_chain.run(classification_results=classification_results_string).strip()

print("\n最終的なグルーピング結果:")
print(grouping_result)


最終的なグルーピング結果:
会議・ミーティング: 定例ミーティング, チーム会議, プロジェクト打合せ, クライアントミーティング, 進捗確認会, プレゼンテーション準備, 部門会議, 月次報告会, オンラインセミナー, 社内研修, 一対一面談, ランチミーティング, 顧客訪問, ワークショップ, ブレインストーミング, 戦略会議, 資料作成ミーティング, 新規企画打合せ, 成果発表会, 取締役会議, 予約確認会, 予算調整会, 業務改善会議, 取引先打合せ, 朝


In [61]:
from collections import Counter

# グループごとのカウント
category_counts = Counter(result['category'] for result in results)
print("\nグループごとのカウント:")
for category, count in category_counts.items():
    print(f"{category}: {count}")


グループごとのカウント:
会議・ミーティング: 29
その他: 1
